In [10]:
import pandas as pd

# 1) Veri yükleme
df_disease_symptoms = pd.read_csv('C:/Users/LENOVO/Desktop/patient_assistant/data/raw/clinical_conditions_symptoms/DiseaseAndSymptoms.csv')
df_disease_precautions = pd.read_csv('C:/Users/LENOVO/Desktop/patient_assistant/data/raw/clinical_conditions_symptoms/Disease precaution.csv')

# 2) Semptom ve önlem sütun isimleri
symptom_cols = [f'Symptom_{i}' for i in range(1, 18)]
precaution_cols = [f'Precaution_{i}' for i in range(1, 5)]

# 3) Hastalık başına semptom listesini çıkar (küçük harfe çevir ve null atla)
def get_symptoms(row):
    return [str(s).strip().lower() for s in row[symptom_cols].values if pd.notna(s)]

df_disease_symptoms['symptoms_list'] = df_disease_symptoms.apply(get_symptoms, axis=1)

# 4) Hastalık başına önlem listesini çıkar
def get_precautions(row):
    return [str(p).strip().lower() for p in row[precaution_cols].values if pd.notna(p)]

df_disease_precautions['precautions_list'] = df_disease_precautions.apply(get_precautions, axis=1)

# 5) İki veriseti birleştirilir (önce tekrarları sil)
df_symptoms_unique = df_disease_symptoms[['Disease', 'symptoms_list']].drop_duplicates(subset='Disease')
df_precautions_unique = df_disease_precautions[['Disease', 'precautions_list']].drop_duplicates(subset='Disease')

df_full = pd.merge(
    df_symptoms_unique,
    df_precautions_unique,
    on='Disease',
    how='inner'
)

# 6) Tahmin fonksiyonu
def predict_disease(user_input):
    # Kullanıcıdan gelen semptomları liste haline getir
    input_symptoms = [s.strip().lower() for s in user_input.split(',')]
    
    # Her hastalık için kullanıcı semptomları ile kaçının eşleştiğini say
    df_full['match_count'] = df_full['symptoms_list'].apply(
        lambda disease_symptoms: len(set(disease_symptoms) & set(input_symptoms))
    )
    
    # En az 1 semptom eşleşenleri al, sonra azalan sırada sırala
    results = df_full[df_full['match_count'] > 0].sort_values(by='match_count', ascending=False)
    
    if results.empty:
        print("Girdiğiniz semptomlarla eşleşen hastalık bulunamadı.")
        return
    
    # En çok eşleşen ilk 5 hastalığı göster
    for idx, row in results.head(5).iterrows():
        print(f"\nTahmini Hastalık: {row['Disease']}")
        print(f"Eşleşen Semptom Sayısı: {row['match_count']}")
        print("Önlemler:")
        for prec in row['precautions_list']:
            print(f" - {prec.capitalize()}")

# 7) Kullanıcıdan interaktif semptom girişi al
if __name__ == '__main__':
    user_symptoms = input("Semptomları virgülle ayırarak girin (örn: fever, cough, headache):\n")
    predict_disease(user_symptoms)


Semptomları virgülle ayırarak girin (örn: fever, cough, headache):
 itching, skin rash, red spots



Tahmini Hastalık: Fungal infection
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Bath twice
 - Use detol or neem in bathing water
 - Keep infected area dry
 - Use clean cloths

Tahmini Hastalık: Chronic cholestasis
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Cold baths
 - Anti itch medicine
 - Consult doctor
 - Eat healthy

Tahmini Hastalık: Drug Reaction
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Stop irritation
 - Consult nearest hospital
 - Stop taking drug
 - Follow up

Tahmini Hastalık: Jaundice
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Drink plenty of water
 - Consume milk thistle
 - Eat fruits and high fiberous food
 - Medication

Tahmini Hastalık: Chicken pox
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Use neem in bathing
 - Consume neem leaves
 - Take vaccine
 - Avoid public places


In [12]:
turkce_to_ingilizce = {
    "ateş": "fever",
    "öksürük": "cough",
    "baş ağrısı": "headache",
    "boğaz ağrısı": "throat irritation",
    "burun akıntısı": "runny nose",
    "kas ağrıları": "muscle pain",
    "vücut ağrısı": "body pain",
    "yorgunluk": "fatigue",
    "bulantı": "nausea",
    "kusma": "vomiting",
    "ishal": "diarrhoea",
    "kabızlık": "constipation",
    "mide ağrısı": "abdominal pain",
    "göğüs ağrısı": "chest pain",
    "nefes darlığı": "breathlessness",
    "göz kızarıklığı": "redness of eyes",
    "gözlerde kaşıntı": "watering from eyes",
    "cilt döküntüsü": "skin rash",
    "kaşıntı": "itching",
    "kırmızı lekeler": "red spots over body",
    "renkli balgam": "phlegm",
    "balgam": "mucous",
    "gözlerde şişlik": "swelled lymph nodes",
    "zihin bulanıklığı": "lack of concentration",
    "iştahsızlık": "loss of appetite",
    "uyku problemleri": "lack of sleep",
    "sersemlik": "dizziness",
    "göğüs daralması": "chest tightness",
    "ağız kuruluğu": "dry mouth",
    "terleme": "sweating",
    "aşırı terleme": "excessive sweating",
    "karın şişliği": "abdominal distention",
    "mide bulantısı": "nausea",
    "hafıza kaybı": "loss of balance",
    "sarılık": "yellowing of eyes",
    "idrar renginde koyulaşma": "dark urine",
    "ağızda acı tat": "bitter taste",
    "idrar yaparken yanma": "burning micturition",
    "sırt ağrısı": "back pain",
    "ellerde titreme": "tremors",
    "nefes almada zorluk": "difficulty in breathing",
    "kas güçsüzlüğü": "weakness of one body side",
    "halsizlik": "weakness in limbs",
    "elde uyuşma": "numbness",
    "burun kanaması": "bloody nose",
    "bayılma": "passage of gases",
    "kasılmalar": "seizures",
    "anemi": "anemia",
    "şişmiş karın": "enlarged abdomen",
    "kilo kaybı": "weight loss",
    "el ve ayaklarda şişlik": "swelling of limbs",
    "ağız kokusu": "bad breath",
    "ciltte kuruluk": "dry skin",
    "kaşıntılı döküntüler": "skin peeling",
    "idrar kaçırma": "bladder discomfort",
    "kaburgalarda ağrı": "pain during bowel movements",
    "vücutta uyuşma": "loss of balance",
    "baygınlık": "blackout",
    "parmaklarda uyuşma": "numbness of fingers",
    "konuşma zorluğu": "slurred speech",
    "çift görme": "visual disturbances",
    "bacak ağrısı": "knee pain",
    "eklem ağrısı": "joint pain",
    "diz şişmesi": "swelling joints"
}


In [14]:
def translate_symptoms(user_input, mapping):
    turkce_list = [s.strip().lower() for s in user_input.split(',')]
    return [mapping.get(s, "") for s in turkce_list if mapping.get(s, "") != ""]


In [16]:
translate_symptoms("baş ağrısı, mide ağrısı, kaşıntı", turkce_to_ingilizce)
# Çıktı: ['headache', 'abdominal pain', 'itching']


['headache', 'abdominal pain', 'itching']

In [18]:
def predict_disease_tr(user_input_tr):
    # Türkçe semptomları İngilizce'ye çevir
    input_symptoms = []
    for s in user_input_tr.split(','):
        s = s.strip().lower()
        if s in turkce_to_ingilizce:
            input_symptoms.append(turkce_to_ingilizce[s])
        else:
            print(f"Uyarı: '{s}' semptomu sözlükte bulunamadı.")

    if not input_symptoms:
        print("Geçerli semptom girişi yapılmadı.")
        return

    # Eşleşme sayısını hesapla
    df_full['match_count'] = df_full['symptoms_list'].apply(
        lambda disease_symptoms: len(set(disease_symptoms) & set(input_symptoms))
    )

    # Eşleşenleri sırala
    results = df_full[df_full['match_count'] > 0].sort_values(by='match_count', ascending=False)

    if results.empty:
        print("Girdiğiniz semptomlarla eşleşen hastalık bulunamadı.")
        return

    # En çok eşleşen ilk 5 hastalığı yazdır
    for idx, row in results.head(5).iterrows():
        print(f"\nTahmini Hastalık: {row['Disease']}")
        print(f"Eşleşen Semptom Sayısı: {row['match_count']}")
        print("Önlemler:")
        for prec in row['precautions_list']:
            print(f" - {prec.capitalize()}")


In [20]:
if __name__ == '__main__':
    user_input_tr = input("Semptomları Türkçe ve virgülle ayırarak girin (örn: baş ağrısı, öksürük, mide ağrısı):\n")
    predict_disease_tr(user_input_tr)


Semptomları Türkçe ve virgülle ayırarak girin (örn: baş ağrısı, öksürük, mide ağrısı):
 ateş, mide ağrısı, baş ağrısı



Tahmini Hastalık: Hypertension 
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Meditation
 - Salt baths
 - Reduce stress
 - Get proper sleep

Tahmini Hastalık: Migraine
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Meditation
 - Reduce stress
 - Use poloroid glasses in sun
 - Consult doctor

Tahmini Hastalık: Paralysis (brain hemorrhage)
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Massage
 - Eat healthy
 - Exercise
 - Consult doctor

Tahmini Hastalık: Malaria
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Consult nearest hospital
 - Avoid oily food
 - Avoid non veg food
 - Keep mosquitos out

Tahmini Hastalık: Chicken pox
Eşleşen Semptom Sayısı: 1
Önlemler:
 - Use neem in bathing
 - Consume neem leaves
 - Take vaccine
 - Avoid public places


In [24]:
symptom_translate = {
    "itching": "kaşıntı",
    "skin rash": "cilt döküntüsü",
    "nodal skin eruptions": "nodüler cilt döküntüleri",
    "continuous sneezing": "sürekli hapşırma",
    "shivering": "titreme",
    "chills": "üşüme",
    "joint pain": "eklem ağrısı",
    "stomach pain": "mide ağrısı",
    "acidity": "asitlik",
    "ulcers on tongue": "dilde ülser",
    "muscle wasting": "kas erimesi",
    "vomiting": "kusma",
    "burning micturition": "idrar yaparken yanma",
    "spotting urination": "lekeli idrara çıkma",
    "fatigue": "yorgunluk",
    "weight gain": "kilo alımı",
    "anxiety": "anksiyete",
    "cold hands and feets": "soğuk eller ve ayaklar",
    "mood swings": "ruh hali değişimleri",
    "weight loss": "kilo kaybı",
    "restlessness": "huzursuzluk",
    "lethargy": "uyuşukluk",
    "patches in throat": "boğazda beyaz lekeler",
    "irregular sugar level": "düzensiz şeker seviyesi",
    "cough": "öksürük",
    "high fever": "yüksek ateş",
    "sunken eyes": "çökük gözler",
    "breathlessness": "nefes darlığı",
    "sweating": "terleme",
    "dehydration": "susuz kalma",
    "indigestion": "hazımsızlık",
    "headache": "baş ağrısı",
    "yellowish skin": "sararmış cilt",
    "dark urine": "koyu renkli idrar",
    "nausea": "bulantı",
    "loss of appetite": "iştah kaybı",
    "pain behind the eyes": "göz arkası ağrı",
    "back pain": "sırt ağrısı",
    "constipation": "kabızlık",
    "abdominal pain": "karın ağrısı",
    "diarrhoea": "ishal",
    "mild fever": "hafif ateş",
    "yellow urine": "sarı idrar",
    "yellowing of eyes": "gözlerde sararma",
    "acute liver failure": "akut karaciğer yetmezliği",
    "fluid overload": "sıvı yüklenmesi",
    "swelling of stomach": "karın şişliği",
    "swelled lymph nodes": "şişmiş lenf düğümleri",
    "malaise": "rahatsızlık hissi",
    "blurred and distorted vision": "bulanık ve bozuk görme",
    "phlegm": "balgam",
    "throat irritation": "boğaz tahrişi",
    "redness of eyes": "göz kızarıklığı",
    "sinus pressure": "sinüs basıncı",
    "runny nose": "burun akıntısı",
    "congestion": "burun tıkanıklığı",
    "chest pain": "göğüs ağrısı",
    "weakness in limbs": "uzuvlarda güçsüzlük",
    "fast heart rate": "hızlı kalp atışı",
    "pain during bowel movements": "bağırsak hareketleri sırasında ağrı",
    "pain in anal region": "makat bölgesinde ağrı",
    "bloody stool": "kanlı dışkı",
    "irritation in anus": "makatta tahriş",
    "neck pain": "boyun ağrısı",
    "dizziness": "baş dönmesi",
    "cramps": "kramp",
    "bruising": "morarma",
    "obesity": "obezite",
    "swollen legs": "şişmiş bacaklar",
    "swollen blood vessels": "şişmiş damarlar",
    "puffy face and eyes": "şişmiş yüz ve gözler",
    "enlarged thyroid": "büyümüş tiroid",
    "brittle nails": "kırılgan tırnaklar",
    "swollen extremeties": "şişmiş uzuvlar",
    "excessive hunger": "aşırı açlık",
    "extra marital contacts": "evlilik dışı ilişkiler",
    "drying and tingling lips": "kurumuş ve karıncalanan dudaklar",
    "slurred speech": "peltek konuşma",
    "knee pain": "diz ağrısı",
    "hip joint pain": "kalça eklem ağrısı",
    "swelling joints": "şişmiş eklemler",
    "painful walking": "ağrılı yürüyüş",
    "movement stiffness": "hareket kısıtlılığı",
    "spinning movements": "baş dönmesi hareketi",
    "loss of balance": "denge kaybı",
    "unsteadiness": "dengesizlik",
    "weakness of one body side": "vücudun bir tarafında güçsüzlük",
    "loss of smell": "koku kaybı",
    "bladder discomfort": "mesane rahatsızlığı",
    "foul smell of urine": "kötü kokulu idrar",
    "continuous feel of urine": "sürekli idrara çıkma isteği",
    "passage of gases": "gaz çıkışı",
    "internal itching": "iç kaşıntı",
    "toxic look (typhos)": "toksik görünüm",
    "depression": "depresyon",
    "irritability": "sinirlilik",
    "muscle weakness": "kas güçsüzlüğü",
    "altered sensorium": "bilinç bulanıklığı",
    "red spots over body": "vücutta kırmızı lekeler",
    "belly pain": "karın ağrısı",
    "abnormal menstruation": "düzensiz regl",
    "dischromic patches": "renk değişimi olan lekeler",
    "watering from eyes": "gözlerde yaşarma",
    "increased appetite": "artmış iştah"
}


In [26]:
precaution_translate = {
    "drink plenty of water": "bol su iç",
    "avoid fatty spicy food": "yağlı ve baharatlı yiyeceklerden kaçın",
    "consume milk thistle": "devedikeni tüket",
    "eat high calorie vegetables": "yüksek kalorili sebzeler ye",
    "antibiotic therapy": "antibiyotik tedavisi",
    "consult doctor": "doktora danış",
    "meditation": "meditasyon",
    "salt baths": "tuz banyoları",
    "reduce stress": "stresi azalt",
    "get proper sleep": "yeterince uyu",
    "eat healthy": "sağlıklı beslen",
    "exercise": "egzersiz yap",
    "rest": "dinlen",
    "take vaccine": "aşı ol",
    "avoid public places": "kalabalıklardan uzak dur",
    "consume neem leaves": "neem yaprağı tüket",
    "use neem in bathing": "banyoda neem kullan",
    "use detol or neem in bathing water": "banyo suyuna antiseptik ekle",
    "keep infected area dry": "enfekte bölgeyi kuru tut",
    "use clean cloths": "temiz kıyafet giy",
    "stop irritation": "kaşımayı bırak",
    "stop taking drug": "ilacı bırak",
    "follow up": "takipte kal",
    "cold baths": "soğuk banyo yap",
    "anti itch medicine": "kaşıntı ilacı",
    "avoid oily food": "yağlı yiyeceklerden uzak dur",
    "apply ice": "buz uygula",
    "maintain hygiene": "hijyene dikkat et",
    "keep hydrated": "vücudu susuz bırakma"
}


In [28]:
# Türkçeye çevir
for col in symptom_cols:
    df_disease_symptoms[col] = df_disease_symptoms[col].apply(
        lambda x: symptom_translate.get(str(x).strip().lower(), x) if pd.notna(x) else x
    )


In [ ]:
for col in precaution_cols:
    df_disease_precautions[col] = df_disease_precautions[col].apply(
        lambda x: precaution_translate.get(str(x).strip().lower(), x) if pd.notna(x) else x
    )


In [32]:
df_disease_symptoms.to_csv("C:/Users/LENOVO/Desktop/patient_assistant/data/raw/clinical_conditions_symptoms/tr_disease_symptoms.csv", index=False)
df_disease_precautions.to_csv("C:/Users/LENOVO/Desktop/patient_assistant/data/raw/clinical_conditions_symptoms/tr_disease_precautions.csv", index=False)


In [34]:
import pandas as pd

# Türkçe veri setlerini yükle
df_symptoms = pd.read_csv("C:/Users/LENOVO/Desktop/patient_assistant/data/raw/clinical_conditions_symptoms/tr_disease_symptoms.csv")
df_precautions = pd.read_csv("C:/Users/LENOVO/Desktop/patient_assistant/data/raw/clinical_conditions_symptoms/tr_disease_precautions.csv")

# Semptom ve önlem sütun isimleri (verinize göre kontrol edin)
symptom_cols = [f'Symptom_{i}' for i in range(1, 18)]
precaution_cols = [f'Precaution_{i}' for i in range(1, 5)]

# Semptom listesini çıkar
def get_symptoms(row):
    return [str(s).strip() for s in row[symptom_cols].values if pd.notna(s)]

df_symptoms['symptoms_list'] = df_symptoms.apply(get_symptoms, axis=1)

# Önlem listesini çıkar
def get_precautions(row):
    return [str(p).strip() for p in row[precaution_cols].values if pd.notna(p)]

df_precautions['precautions_list'] = df_precautions.apply(get_precautions, axis=1)

# İki dataframe'i hastalık adına göre birleştir
df_full = pd.merge(
    df_symptoms[['Disease', 'symptoms_list']],
    df_precautions[['Disease', 'precautions_list']],
    on='Disease',
    how='inner'
)

# Tahmin fonksiyonu
def hastalik_tahmin(etken_semptomlar):
    # Kullanıcı semptomlarını liste yap
    input_symptoms = [s.strip().lower() for s in etken_semptomlar.split(',')]
    
    # Her hastalık için eşleşen semptom sayısını hesapla
    df_full['eslesen_sayisi'] = df_full['symptoms_list'].apply(
        lambda hastalik_semptomlari: len(set(hastalik_semptomlari) & set(input_symptoms))
    )
    
    # Eşleşen semptom sayısı 1'den fazla olanları sırala
    results = df_full[df_full['eslesen_sayisi'] > 0].sort_values(by='eslesen_sayisi', ascending=False)
    
    if results.empty:
        print("Girdiğiniz semptomlarla eşleşen hastalık bulunamadı.")
        return
    
    # En çok eşleşen ilk 5 hastalığı yazdır
    for idx, row in results.head(5).iterrows():
        print(f"\nTahmini Hastalık: {row['Disease']}")
        print(f"Eşleşen Semptom Sayısı: {row['eslesen_sayisi']}")
        print("Önlemler:")
        for onlem in row['precautions_list']:
            print(f" - {onlem.capitalize()}")

# Test et
if __name__ == '__main__':
    kullanici_girdisi = input("Semptomları virgülle ayırarak girin (örn: ateş, öksürük, baş ağrısı):\n")
    hastalik_tahmin(kullanici_girdisi)


Semptomları virgülle ayırarak girin (örn: ateş, öksürük, baş ağrısı):
 ateş, kusma, ishal



Tahmini Hastalık: Typhoid
Eşleşen Semptom Sayısı: 2
Önlemler:
 - Eat high calorie vegitables
 - Antiboitic therapy
 - Consult doctor
 - Medication

Tahmini Hastalık: Typhoid
Eşleşen Semptom Sayısı: 2
Önlemler:
 - Eat high calorie vegitables
 - Antiboitic therapy
 - Consult doctor
 - Medication

Tahmini Hastalık: Typhoid
Eşleşen Semptom Sayısı: 2
Önlemler:
 - Eat high calorie vegitables
 - Antiboitic therapy
 - Consult doctor
 - Medication

Tahmini Hastalık: Typhoid
Eşleşen Semptom Sayısı: 2
Önlemler:
 - Eat high calorie vegitables
 - Antiboitic therapy
 - Consult doctor
 - Medication

Tahmini Hastalık: Typhoid
Eşleşen Semptom Sayısı: 2
Önlemler:
 - Eat high calorie vegitables
 - Antiboitic therapy
 - Consult doctor
 - Medication


In [36]:
print(df_precautions.head())


          Disease                      Precaution_1  \
0   Drug Reaction                   stop irritation   
1         Malaria          Consult nearest hospital   
2         Allergy                    apply calamine   
3  Hypothyroidism                     reduce stress   
4       Psoriasis  wash hands with warm soapy water   

                   Precaution_2        Precaution_3  \
0      consult nearest hospital    stop taking drug   
1               avoid oily food  avoid non veg food   
2       cover area with bandage                 NaN   
3                      exercise         eat healthy   
4  stop bleeding using pressure      consult doctor   

                  Precaution_4  \
0                    follow up   
1           keep mosquitos out   
2  use ice to compress itching   
3             get proper sleep   
4                   salt baths   

                                    precautions_list  
0  [stop irritation, consult nearest hospital, st...  
1  [Consult nearest hos